<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-10-28 16:35:56


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 2


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.drop('TTM', axis=1, inplace=True)
df_latest_analysis

HDFCBANK
COLPAL


Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
0,Sales,"38,042","42,381","53,713","62,137","59,248","58,584","77,780",HDFCBANK.NS
1,Profit,"23,393","28,464","34,318","38,195","42,796","50,873","61,498",HDFCBANK.NS
2,Sales,"3,982","4,188","4,462","4,525","4,841","5,100","5,226",COLPAL.NS
3,Profit,851,983,"1,143","1,043","1,350","1,409","1,410",COLPAL.NS


In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='7y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[stock_df['Year'].str.contains("Mar")]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
0,Close,687.00,904.20,1118.79,838.70,1453.45,1436.97,1591.3,HDFCBANK.NS
0,Close,867.56,930.79,1131.42,1149.55,1464.71,1486.02,1488.0,COLPAL.NS


In [6]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
0,Sales,"38,042","42,381","53,713","62,137","59,248","58,584","77,780",HDFCBANK.NS
1,Profit,"23,393","28,464","34,318","38,195","42,796","50,873","61,498",HDFCBANK.NS
2,Sales,"3,982","4,188","4,462","4,525","4,841","5,100","5,226",COLPAL.NS
3,Profit,851,983,"1,143","1,043","1,350","1,409","1,410",COLPAL.NS
0,Close,687.0,904.2,1118.79,838.7,1453.45,1436.97,1591.3,HDFCBANK.NS
0,Close,867.56,930.79,1131.42,1149.55,1464.71,1486.02,1488.0,COLPAL.NS
